<a href="https://colab.research.google.com/github/farhanhubble/discover-drl/blob/master/Rediscovering_RL_Notebook_0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement learning with Foolsball
- Reinforcement learning is learning to make decisions from experience.
- Games are a good testbed for agents to interaction with an environment and explore it.
 

# About Foolsball
- 5x4 playground that provides a football/foosball-like environment.
- An agent or actor:
  - always spawned in the top-left corner
  - displayed as '⚽'
  - can move North, South, East or West.
  - can be controlled algorithm
- A number of **static** opponents that occupy certain locations on the field.
- A goalpost that is fixed in the bottom right corner

## Primary goal
- We want the agent to learn to reach the goalpost 

## Secondary goals
- We may want the agent to learn to be efficient in some sense, for example, take the shortest path to the goalpost. **More precisely we want an algorithm to learn to control the agent and steer it towards the goalpost.**

In [1]:
agent = '⚽'
opponent = '👕'
goal = '🥅'

arena = [['⚽', ' ' , '👕', ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , '🥅']]

# Implementing an environment for the game of Foolsball
- OpenAI Gym has many [text environments](https://github.com/openai/gym/tree/master/gym/envs/toy_text)
- Text environments are simple to render in a notebook and super-fast to experiment with.
- We want to build out own environment for two reasons:
  - It's a great exercise in understanding the finer details, like states, actions, rewards, returns.
  - Some of the experimentation we do requires looking under the hood of the environment, which is easier with your own implementation than OpenAI Gym.
  - OpenAI Gym has a simple `step(), reset()` API that we also implement. So porting our implentation over to Gym shoud be easy (and fun)!



# Understanding the first bits of terminology.
## State 
- In RL state refers to information about the environemnt and then agent.
- An RL algorithm inspects the state to decide which action to take.
- Exactly what information gets captured in `state` depends on a few factors:
  - The complexity of the environment: 
    - The number of actors, 
    - the nature of the environment, for example text or images. 
  - The complexity of the algorithm
    - A simple algorithm may only need information about the agent and its immediate surroundings.
    - A more complex algorithm may need information about the whole environment.


## Our setup
- In our case we want the algorithm to only know about the location of the agent on the field. 
- We could have included information about the opponents too which would perhaps aid in the decision making but we chose not to.  

In [2]:
import numpy as np

class Foolsball(object):

  def __to_state__(self,row,col):
    """Convert from integer state to indices (row,col)."""
    return row*self.n_cols + col

  def __to_indices__(self, state):
    """Convert indices(row,col) to state (single integer)."""
    row = state // self.n_cols
    col = state % self.n_cols
    return row,col

  def __deserialize__(self,map:list,agent:str,opponent:str, goal:str):
    """Convrt a string representation of a map into a 2D numpy array
    Param map: list of lists of strings representing the player, opponents and goal.
    Param agent: string representing the agent on the map 
    Param opponent: string representing every instance of an opponent player
    Param goal: string representing the location of the goal on the map
    """
    ## Capture dimensions and map.
    self.n_rows = len(map)
    self.n_cols = len(map[0])
    self.n_states = self.n_rows * self.n_cols 
    self.map = np.asarray(map)

    ## Store string representations for printing the map, etc.
    self.agent_repr = agent
    self.opponent_repr  = opponent
    self.goal_repr = goal

    ## Find initial state, the desired goal state and the state of the opponents. 
    self.init_state = None
    self.goal_state = None
    self.opponents_states = []

    for row in range(self.n_rows):
      for col in range(self.n_cols):
        if map[row][col] == agent:
          # Store the initial state outside the map.
          # This helps in quickly resetting the game to the initial state and
          # also simplifies printing the map independent of the agent's state. 
          self.init_state = self.__to_state__(row,col)
          self.map[row,col] = ' ' 
        
        elif map[row][col] == opponent:
          self.opponents_states.append(self.__to_state__(row,col))

        elif map[row][col] == goal:
          self.goal_state = self.__to_state__(row,col)

    assert self.init_state is not None, print(f"Map {map} does not specify an agent {agent} location")
    assert self.goal_state is not None,  print(f"Map {map} does not specify a goal {goal} location")
    assert self.opponents_states,  print(f"Map {map} does not specify any opponents {opponent} location")

    return self.init_state


  def __get_next_state_on_action__(self,state,action):
    """Return next state based on current state and action."""
    row, col = self.__to_indices__(state)
    action_to_index_delta = {'n':[-1,0], 'e':[0,+1], 'w':[0,-1], 's':[+1,0]}

    row_delta, col_delta = action_to_index_delta[action]
    new_row , new_col = row+row_delta, col+col_delta

    ## Return current state if next state is invalid
    ## The caller need to check for this
    if not(0<=new_row<self.n_rows) or not(0<=new_col<self.n_cols):
      return state  
    
    return self.__to_state__(new_row, new_col)


  def __get_reward_for_transition__(self,state,next_state):
    """ Return the reward based on the transition from current state to next state. """
    ## Transition rejected due to illegal action (move)
    if next_state == self.state:
      reward = self.rewards['outside']
    
    ## Goal!
    elif next_state == self.goal_state:
      reward = self.rewards['goal']
    
    ## Ran into opponent. 
    elif next_state in self.opponents_states:
      reward = self.rewards['opponent']

    ## Made a safe and valid move.   
    else:
      reward = self.rewards['unmarked']

    return reward


  def __is_terminal_state__(self, state):
    return (state == self.goal_state) or (state in self.opponents_states) 

  
  def __init__(self,map,agent,opponent,goal):
    """Spawn the world, create variables to track state and actions."""
    # We just need to track the location of the agent (the ball)
    # Everything else is static and so a potential algorithm doesn't 
    # have to look at it. The `done` flag terminal states.
    self.state = self.__deserialize__(map,agent,opponent,goal)
    self.done = False
    self.actions = ['n','e','w','s']

    # Build a trnasition table (dict of dicts), mapping every (state,action)
    # pair to the next state. This defines the one-step dynamics of the 
    # environment 
    self.transitions = self.__install_transition_table__()


    # Set up the rewards
    self.default_rewards = {'unmarked':-1, 'opponent':-5, 'outside':-1, 'goal':+5}
    self.set_rewards(self.default_rewards)

  
  def __install_transition_table__(self):
    ## Create a dictionary of dictionaries that can map every (state,action) pair
    ## to the corresponding next state.
    transitions = {s:{a:None for a in self.actions} for s in range(self.n_states)}
    for s in range(self.n_states):
      for a in self.actions:
        transitions[s][a] = self.__get_next_state_on_action__(s,a)


  def reset(self):
    """Reset the environment to its initial state."""
    # There's really just two things we need to reset: the state, which should
    # be reset to the initial state, and the `done` flag which should be 
    # cleared to signal that we are not in a terminal state anymore, even if we 
    # were earlier. 
    self.state = self.init_state
    self.done  = False
    return self.state

  
  def set_rewards(self,rewards):
    if not self.state == self.init_state:
      print('Warning: Setting reward while not in initial state! You may want to call reset() first.')
    for key in self.default_rewards:
      assert key in rewards, print(f'Key {key} missing from reward.') 
    self.rewards = rewards

  
  def step(self,action):
    """Simulate state transition based on current state and action received."""
    assert not self.done, \
    print(f'You cannot call step() in a terminal state({self.state}). Check the "done" flag before calling step() to avoid this.')
    next_state = self.__get_next_state_on_action__(self.state, action)

    reward = self.__get_reward_for_transition__(self.state, next_state)
    done = self.__is_terminal_state__(next_state)

    self.state, self.done = next_state, done
    
    return next_state, reward, done


  def render(self):
    """Pretty-print the environment and agent."""
    ## Create a copy of the map and change data type to accomodate
    ## 3-character strings
    _map = np.array(self.map, dtype='<U3')

    ## Mark unoccupied positions with special symbol.
    ## And add extra spacing to align all columns.
    for row in range(_map.shape[0]):
      for col in range(_map.shape[1]):
        if _map[row,col] == ' ':
          _map[row,col] = ' + '
        
        elif _map[row,col] == self.opponent_repr: 
          _map[row,col] =  self.opponent_repr + ' '
        
        elif _map[row,col] == self.goal_repr:
          _map[row,col] = ' ' + self.goal_repr + ' '
      
    ## If current state overlaps with the goal state or one of the opponents'
    ## states, susbstitute a distinct marker.
    if self.state == self.goal_state:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' 🏁 '
    elif self.state in self.opponents_states:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' ❗ '
    else:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' ' + self.agent_repr
    
    for row in range(_map.shape[0]):
      for col in range(_map.shape[1]):
        print(f' {_map[row,col]} ',end="")
      print('\n') 
    
    print()



In [3]:
foolsball = Foolsball(arena, agent, opponent, goal)

In [4]:
foolsball.render()

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  




In [5]:
## Move: n,s,e,w
## Reset: r
## Exit: x
while True:
  try:
    act = input('>>')

    if act in foolsball.actions:
      print(foolsball.step(act))
      print()
      foolsball.render()
    elif act == 'r':
      print(foolsball.reset())
      print()
      foolsball.render()
    elif act == 'x':
      break
    else:
      print(f'Invalid input:{act}')
  except Exception as e:
    print(e)

>>x


In [6]:
## Reward and return
path1 = ['e','s','e','s','s','s','e']
path2 = ['s','e','e','s','s','s','e']
path3 = ['s','s','s','e','e','s','e']
path4 = ['s','s','s','s','n','e','e','s','e']

In [7]:
def get_return(path):
  foolsball.reset()
  foolsball.render()
  _return_ = 0
  for act in path: 
    next_state, reward, done = foolsball.step(act)
    foolsball.render()
    _return_ += reward
    
    if done:
      break
    
  print(f'Return (accumulated reward): {_return_}')

In [8]:
get_return(path1)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    ⚽  👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    ⚽   +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    ⚽  👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    ⚽   +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): -1


In [9]:
get_return(path2)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  ⚽   +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    ⚽   +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    ⚽  👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    ⚽   +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): -1


In [10]:
get_return(path3)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  ⚽   +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  ⚽  👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    ⚽   +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): -1


In [11]:
get_return(path4)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  ⚽   +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  ⚽  👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  ⚽  👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    ⚽   +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    + 

In [12]:
## Different reward structure
foolsball.set_rewards({'unmarked':0, 'opponent':-5, 'outside':-1, 'goal':+5})

In [13]:
get_return(path1)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    ⚽  👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    ⚽   +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    ⚽  👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    ⚽   +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): 5


In [14]:
get_return(path4)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  ⚽   +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  ⚽  👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  ⚽  👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    ⚽   +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    + 

In [15]:
def get_discounted_return(path, gamma=0):
  foolsball.reset()
  foolsball.render()
  _return_ = 0
  discount_coeff = 1
  for act in path: 
    next_state, reward, done = foolsball.step(act)
    foolsball.render()
    _return_ += discount_coeff*reward
    discount_coeff *= gamma    
    if done:
      break
    
  print(f'Return (accumulated reward): {_return_}')

In [16]:
HYPER_PARAMS = {'gamma':0.9}

In [17]:
get_discounted_return(path1, HYPER_PARAMS['gamma'])

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    ⚽  👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    ⚽   +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    ⚽  👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    ⚽   +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): 2.6572050000000007


In [18]:
get_discounted_return(path4, HYPER_PARAMS['gamma'])

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  ⚽   +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  ⚽  👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  ⚽  👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    ⚽   +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    + 

In [19]:
## Highest return = max(return_path1, return_path2)

In [20]:
## Selecting a path is making a sequence of decisions.
## Enumerating all paths is a catastrophic combinatorial explosion.
## The trick is to calculate the action with the highest **return** for every state.

In [21]:
import pandas as pd

In [22]:
REWARDS_TBL = pd.DataFrame.from_dict({s:{a:None for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
REWARDS_TBL

,n,e,w,s
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,None,None,None,None
8,None,None,None,None
9,None,None,None,None


In [23]:
for state in REWARDS_TBL.index:
  if not foolsball.__is_terminal_state__(state):
    for action in REWARDS_TBL.columns:
      next_state = foolsball.__get_next_state_on_action__(state,action)
      REWARDS_TBL.loc[state, action] = foolsball.__get_reward_for_transition__(state, next_state)

In [24]:
terminal_states = foolsball.opponents_states+[foolsball.goal_state]
print(terminal_states)
REWARDS_TBL

[2, 7, 9, 15, 17, 19]


,n,e,w,s
0,0,0,0,0
1,0,-5,0,0
2,None,None,None,None
3,0,0,-5,-5
4,0,0,0,0
5,0,0,0,-5
6,-5,-5,0,0
7,None,None,None,None
8,0,-5,0,0
9,None,None,None,None


In [25]:
RETURNS_TBL = pd.DataFrame.from_dict({s:{a:None for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

In [26]:
RETURNS_TBL

,n,e,w,s
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,None,None,None,None
8,None,None,None,None
9,None,None,None,None


In [27]:
RETURNS_TBL.loc[terminal_states]

,n,e,w,s
2,None,None,None,None
7,None,None,None,None
9,None,None,None,None
15,None,None,None,None
17,None,None,None,None
19,None,None,None,None


In [28]:
RETURNS_TBL.loc[terminal_states] = 0

In [29]:
RETURNS_TBL

,n,e,w,s
0,None,None,None,None
1,None,None,None,None
2,0,0,0,0
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


In [30]:
def make_returns_table(terminal_states):
  table = pd.DataFrame.from_dict({s:{a:None for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
  table.loc[terminal_states] = 0
  return table

In [31]:
def fill_returns_table_v0(table,state,debug=False): 
  for action in table.columns:
    if table.loc[state][action] is None:
      next_state = foolsball.__get_next_state_on_action__(state, action)
      reward = foolsball.__get_reward_for_transition__(state, next_state)

      if debug:
        print(f'Trying to fill ({state},{action},{next_state})')
      
      fill_returns_table_v0(table, next_state, debug) # <= Earth shaking problem here!!! 😱😱😱
      table.loc[state][action]  = reward + HYPER_PARAMS['gamma'] * table.loc[next_state].max()
    
    else:
      if debug:
        print((state,action),f'already has a RETURN {table.loc[state][action]}')
  

In [32]:
table = make_returns_table(terminal_states)
fill_returns_table_v0(table,state=0)

RecursionError: ignored

In [33]:
def fill_returns_table_v1(table,state,debug=False):
  for action in table.columns:
    if table.loc[state][action] is None:
      next_state = foolsball.__get_next_state_on_action__(state, action)
      reward = foolsball.__get_reward_for_transition__(state, next_state)
      
      if debug:
        print(f'Trying to fill ({state},{action},{next_state})')
      
      if next_state == state:
        table.loc[state][action] = -np.inf # <= No self recursion
      else:
        fill_returns_table_v1(table,next_state,debug)
        table.loc[state][action]  = reward + HYPER_PARAMS['gamma'] * table.loc[next_state].max()
    else:
      if debug:
        print((state,action),f'already has a RETURN {table.loc[state][action]}')

In [34]:
table = make_returns_table(terminal_states)
fill_returns_table_v1(table, state=0, debug=False)

RecursionError: ignored

In [35]:
def fill_returns_table_v2(table,state, debug=False):
  for action in table.columns:
    if table.loc[state][action] is None:
      next_state = foolsball.__get_next_state_on_action__(state, action)
      reward = foolsball.__get_reward_for_transition__(state, next_state)
      
      if debug:
        print(f'Trying to fill ({state},{action},{next_state})')
      
      if next_state == state:
        table.loc[state][action] = -np.inf # <= No self recursion
      
      elif not table.loc[next_state].isna().any(): # <= No recursion beyond immediate neighbor!
        table.loc[state][action]  = reward + HYPER_PARAMS['gamma'] * table.loc[next_state].max()
    
    else:
      if debug:
        print((state,action),f'already has a RETURN {table.loc[state][action]}')

In [36]:
table = make_returns_table(terminal_states)

In [37]:
fill_returns_table_v2(table,state=0)

In [38]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,None,None,None,None
2,0,0,0,0
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


In [42]:
fill_returns_table_v2(table,state=1)

In [43]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


In [45]:
fill_returns_table_v2(table,state=3)

In [46]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


In [47]:
for s in range(4,19):
  fill_returns_table_v2(table,state=s)

In [48]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,-inf,None
5,None,None,None,-5
6,-5,-5,None,None
7,0,0,0,0
8,None,-5,-inf,None
9,0,0,0,0


In [49]:
table.isna().sum()

n    8
e    6
w    6
s    8
dtype: int64

In [50]:
for s in range(0,19):
  fill_returns_table_v2(table,state=s)

In [51]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,-inf,None
5,None,None,None,-5
6,-5,-5,None,None
7,0,0,0,0
8,None,-5,-inf,None
9,0,0,0,0


In [52]:
table.isna().sum()

n    8
e    6
w    6
s    8
dtype: int64

In [53]:
for s in range(0,19):
  fill_returns_table_v2(table,state=s)

In [54]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,-inf,None
5,None,None,None,-5
6,-5,-5,None,None
7,0,0,0,0
8,None,-5,-inf,None
9,0,0,0,0


In [55]:
table.isna().sum()

n    8
e    6
w    6
s    8
dtype: int64

## Estimating returns through simulation
- A.k.a Monte Carlo sampling
- No more cheating by peeping into the environment (private APIs)

In [56]:
def collect_random_episode():
  state = foolsball.reset()
  done = False
  episode = []

  while not done:
    action = np.random.choice(foolsball.actions)
    next_state, reward, done = foolsball.step(action)
    episode.append([state, action, reward])
    state = next_state
  
  return episode

In [57]:
ep = collect_random_episode()
foolsball.render()
print(ep)

  +    +    ❗    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


[[0, 'n', -1], [0, 'e', 0], [1, 'e', -5]]


In [58]:
list(zip(*ep))

[(0, 0, 1), ('n', 'e', 'e'), (-1, 0, -5)]

In [59]:
def discounted_return_from_episode(ep, gamma=0):
  states, actions, rewards = list(zip(*ep))
  rewards = np.asarray(rewards)
  discount_coeffs = np.asarray([np.power(gamma,p) for p in range(len(rewards))])
  
  l = len(rewards)
  discounted_returns = [np.dot(rewards[i:],discount_coeffs[:l-i]) for i in range(l)]

  return (states, actions, discounted_returns)


In [60]:
discounted_return_from_episode(ep, gamma=HYPER_PARAMS['gamma'])

((0, 0, 1), ('n', 'e', 'e'), [-5.050000000000001, -4.5, -5.0])

In [61]:
## Estimate returns by simulating lot of episodes.

# Create empty returns table 
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 100  #Try 100, 500, 2000

for i in range(n_episodes):
  episode_i = collect_random_episode()
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += ret
    VISITS_COUNTS_TBL.loc[s,a] += 1
  

In [62]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
estimated_returns

,n,e,w,s
0,-4.759026,-3.927140,-4.736747,-3.611265
1,-4.897472,-4.857143,-4.205653,-3.773605
2,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000
4,-3.788258,-3.466848,-4.599561,-3.939235
5,-3.897634,-3.880526,-3.482536,-4.807692
6,-4.166667,-4.545455,-3.878126,-3.370721
7,0.000000,0.000000,0.000000,0.000000
8,-3.552781,-4.705882,-4.613445,-3.586519
9,0.000000,0.000000,0.000000,0.000000


In [15]:
### Policies!!

In [71]:
def collect_greedy_episode_from_returns_tbl(table, max_ep_len=20):
  state = foolsball.reset()
  done = False
  episode = []

  for _ in range(max_ep_len):
    if done:
      break
    
    greedy_action_index = table.loc[state].argmax()
    greedy_action = table.columns[greedy_action_index]
    next_state, reward, done = foolsball.step(greedy_action)
    episode.append([state, greedy_action, reward])
    state = next_state
  
  return episode

In [72]:
collect_greedy_episode_from_returns_tbl(estimated_returns)

[[0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1],
 [0, 'n', -1]]

In [68]:
def collect_epsilon_greedy_episode_from_returns_tbl(table, max_ep_len=20, epsilon=0.1):
  
  state = foolsball.reset()
  done = False
  episode = []

  for _ in range(max_ep_len):
    if done:
      break
    
    actions = table.columns
    action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)
    
    greedy_action_index = table.loc[state].argmax()
    action_probs[greedy_action_index] += 1-epsilon
    
    epsilon_greedy_action = np.random.choice(table.columns,p=action_probs)

    next_state, reward, done = foolsball.step(epsilon_greedy_action)
    episode.append([state, epsilon_greedy_action, reward])
    state = next_state

  return episode

In [69]:
collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns, epsilon=1)

[[0, 's', 0],
 [4, 'w', -1],
 [4, 's', 0],
 [8, 'n', 0],
 [4, 's', 0],
 [8, 'w', -1],
 [8, 's', 0],
 [12, 's', 0],
 [16, 'w', -1],
 [16, 'e', -5]]

**Q:** What's the problem with this approach?

**A:** We are improving our estimate of the returns but we are not using the estimates when genrating new samples. 

In [73]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 1000

for i in range(n_episodes):
  estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1)
  
  episode_i = collect_greedy_episode_from_returns_tbl(estimated_returns)
  #print(episode_i)
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += ret
    VISITS_COUNTS_TBL.loc[s,a] += 1

In [74]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
estimated_returns

,n,e,w,s
0,-5.759138,-3.892117,-5.759138,0.0
1,-5.607883,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
5,0.000000,0.000000,0.000000,0.0
6,0.000000,0.000000,0.000000,0.0
7,0.000000,0.000000,0.000000,0.0
8,0.000000,0.000000,0.000000,0.0
9,0.000000,0.000000,0.000000,0.0


In [75]:
def greedy_policy_from_returns_tbl(table):
  policy = {s:None for s in table.index }

  for state in table.index:
    if state not in terminal_states:
      greedy_action_index = table.loc[state].argmax()
      greedy_action = table.columns[greedy_action_index]
      policy[state] = greedy_action

  return policy

In [76]:
policy1 = greedy_policy_from_returns_tbl(estimated_returns)
policy1

{0: 's',
 1: 'e',
 2: None,
 3: 'n',
 4: 'n',
 5: 'n',
 6: 'n',
 7: None,
 8: 'n',
 9: None,
 10: 'n',
 11: 'n',
 12: 'n',
 13: 'n',
 14: 'n',
 15: None,
 16: 'n',
 17: None,
 18: 'n',
 19: None}

In [77]:
def pretty_print_policy(policy):
  direction_repr = {'n':' 🡑 ', 'e':' 🡒 ', 'w':' 🡐 ', 's':' 🡓 ', None:' ⬤ '}

  for row in range(foolsball.n_rows):
    for col in range(foolsball.n_cols):
      state = row * foolsball.n_cols + col
      print(direction_repr[policy[state]],end='')
    print()

In [78]:
pretty_print_policy(policy1)

 🡓  🡒  ⬤  🡑 
 🡑  🡑  🡑  ⬤ 
 🡑  ⬤  🡑  🡑 
 🡑  🡑  🡑  ⬤ 
 🡑  ⬤  🡑  ⬤ 


In [79]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 1000

for i in range(n_episodes):
  estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1)
  
  episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns)
  #print(episode_i)
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += ret
    VISITS_COUNTS_TBL.loc[s,a] += 1

In [80]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
estimated_returns

,n,e,w,s
0,-1.586109,-0.324697,-1.306969,-0.223033
1,-1.641446,-4.375000,-0.209992,-0.341340
2,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000
4,-0.221933,-0.324119,-1.481402,-0.536915
5,-0.314328,-2.107500,-0.197914,-4.375000
6,-2.500000,-2.500000,-0.900000,-2.025000
7,0.000000,0.000000,0.000000,0.000000
8,-0.580221,-4.750000,-1.832138,-0.313193
9,0.000000,0.000000,0.000000,0.000000


In [81]:
policy2 = greedy_policy_from_returns_tbl(estimated_returns)
policy2

{0: 's',
 1: 'w',
 2: None,
 3: 'n',
 4: 'n',
 5: 'w',
 6: 'w',
 7: None,
 8: 's',
 9: None,
 10: 'w',
 11: 'e',
 12: 'n',
 13: 'w',
 14: 'w',
 15: None,
 16: 'n',
 17: None,
 18: 'n',
 19: None}

In [82]:
pretty_print_policy(policy2)

 🡓  🡐  ⬤  🡑 
 🡑  🡐  🡐  ⬤ 
 🡓  ⬤  🡐  🡒 
 🡑  🡐  🡐  ⬤ 
 🡑  ⬤  🡑  ⬤ 


### Epsilon-greedy policy
**Guiding Principle:** Greedy policies are bad in the beginning. Random policies in the beginning followed by greedy policies later on crush it!

In [83]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 10000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999

for i in range(n_episodes):
  estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1)
  
  epsilon = max(epsilon,min_epsilon)
  episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns,epsilon=epsilon)
  epsilon *= epsilon_decay
  #print(episode_i)
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += ret
    VISITS_COUNTS_TBL.loc[s,a] += 1

In [84]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
print(estimated_returns)

policy3 = greedy_policy_from_returns_tbl(estimated_returns)
print(policy3)

pretty_print_policy(policy3)

           n         e         w         s
0  -1.674881 -0.834913 -1.700660 -0.282193
1  -3.186662 -4.980620 -0.564936 -1.968554
2   0.000000  0.000000  0.000000  0.000000
3   0.000000  0.000000  0.000000  0.000000
4  -0.270735 -0.892477 -1.575465 -0.798977
5  -2.153700 -1.564351 -0.637848 -4.980843
6  -4.861111 -4.878049 -2.603923 -0.279392
7   0.000000  0.000000  0.000000  0.000000
8  -0.500626 -4.976636 -2.593333 -1.638424
9   0.000000  0.000000  0.000000  0.000000
10 -1.967619 -2.448626 -4.772727  0.930031
11 -4.000000 -4.237583 -1.019127 -4.285714
12 -1.331855 -2.521579 -2.722033 -2.506402
13 -4.000000 -0.477519 -2.470214 -4.642857
14 -1.562400 -4.736842 -0.633355  2.887692
15  0.000000  0.000000  0.000000  0.000000
16 -1.777959 -4.687500 -3.836380 -3.326502
17  0.000000  0.000000  0.000000  0.000000
18  1.540974  4.928571 -4.583333  0.315000
19  0.000000  0.000000  0.000000  0.000000
{0: 's', 1: 'w', 2: None, 3: 'n', 4: 'n', 5: 'w', 6: 's', 7: None, 8: 'n', 9: None, 10: 's', 11: 

## Constant alpha

In [85]:
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 10000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999

alpha = 0.01

for i in range(n_episodes):
  estimated_returns = ESTIMATED_RETURNS_TBL
  
  epsilon = max(epsilon,min_epsilon)
  episode_i = collect_epsilon_greedy_episode_from_returns_tbl(estimated_returns,epsilon=epsilon)
  epsilon *= epsilon_decay
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += alpha*(ret - ESTIMATED_RETURNS_TBL.loc[s,a])

In [86]:
estimated_returns = ESTIMATED_RETURNS_TBL
print(estimated_returns)

policy4 = greedy_policy_from_returns_tbl(estimated_returns)
print(policy4)

pretty_print_policy(policy4)

           n         e         w         s
0   0.219254  0.950220  0.316226  1.514849
1  -2.502203 -4.499471  1.573112 -1.550079
2   0.000000  0.000000  0.000000  0.000000
3   0.000000  0.000000  0.000000  0.000000
4   1.049223  1.799945  0.267699  0.979810
5   0.950002  2.148961  1.203542 -4.931578
6  -4.388036 -4.551857  0.764184  2.796845
7   0.000000  0.000000  0.000000  0.000000
8   1.330195 -3.881557 -1.890940 -0.476121
9   0.000000  0.000000  0.000000  0.000000
10  1.741855  1.933923 -4.394156  3.564899
11 -0.523309 -0.154004  2.622020 -0.612395
12 -1.113745  0.090022 -1.225057 -1.130686
13 -1.031929  3.053475 -0.225365 -0.910465
14  2.680166 -4.316500  2.334727  4.153221
15  0.000000  0.000000  0.000000  0.000000
16 -0.425090 -0.827431 -0.966361 -0.594198
17  0.000000  0.000000  0.000000  0.000000
18  2.754976  5.000000 -4.316500  2.591217
19  0.000000  0.000000  0.000000  0.000000
{0: 's', 1: 'w', 2: None, 3: 'n', 4: 'e', 5: 'e', 6: 's', 7: None, 8: 'n', 9: None, 10: 's', 11: 